In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="yAGqrP4bbcFc0kgTqBks")
project = rf.workspace("aarhus-universitet-r1lgs").project("deep-learning-wnuyb")
version = project.version(12)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Deep-Learning-12 in yolov11:: 100%|██████████| 1128/1128 [00:00<00:00, 2943.37it/s]


In [22]:
from ultralytics import YOLO
import torch

# Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = model.to(device)
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="Deep-Learning-12/data.yaml", epochs=10)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
#results = model("https://ultralytics.com/images/bus.jpg")


engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=Deep-Learning-12/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=F

train: Scanning C:\Users\Bruger\OneDrive\Dokumenter\PyJects\Project\yolov5\datasets\Deep-Learning-12\train\labels.cache... 561 images, 0 backgrounds, 0 corrupt: 100%|██████████| 561/561 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 822. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\Bruger\OneDrive\Dokumenter\PyJects\Project\yolov5\datasets\Deep-Learning-12\valid\labels.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train6
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


10 epochs completed in 0.022 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 5.5MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics 8.3.39  Python-3.12.2 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,586,052 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 44.54it/s]


                   all          1          4      0.986          1      0.995       0.94
                 aeble          1          4      0.986          1      0.995       0.94
Speed: 0.0ms preprocess, 18.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train6
Ultralytics 8.3.39  Python-3.12.2 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,586,052 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\Bruger\OneDrive\Dokumenter\PyJects\Project\yolov5\datasets\Deep-Learning-12\valid\labels.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.15s/it]


                   all          1          4      0.986          1      0.995       0.94
                 aeble          1          4      0.986          1      0.995       0.94
Speed: 3.0ms preprocess, 27.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\detect\train62


In [29]:
results = model.predict(source="data/images",conf=0.2)



image 1/1 c:\Users\Bruger\OneDrive\Dokumenter\PyJects\Project\yolov5\data\images\lettuce_test.jpg: 512x640 1 hakket oksekod, 1 salat, 26.0ms
Speed: 0.0ms preprocess, 26.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)


In [35]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    print(boxes)
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 7., 17.], device='cuda:0')
conf: tensor([0.32103, 0.27564], device='cuda:0')
data: tensor([[4.19142e+01, 4.11390e+00, 6.25132e+02, 5.31442e+02, 3.21026e-01, 7.00000e+00],
        [2.90360e+01, 3.79087e+00, 6.29693e+02, 5.30580e+02, 2.75636e-01, 1.70000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (538, 674)
shape: torch.Size([2, 6])
xywh: tensor([[333.52310, 267.77808, 583.21783, 527.32837],
        [329.36441, 267.18546, 600.65680, 526.78912]], device='cuda:0')
xywhn: tensor([[0.49484, 0.49773, 0.86531, 0.98016],
        [0.48867, 0.49663, 0.89118, 0.97916]], device='cuda:0')
xyxy: tensor([[ 41.91417,   4.11390, 625.13202, 531.44226],
        [ 29.03601,   3.79087, 629.69281, 530.58002]], device='cuda:0')
xyxyn: tensor([[0.06219, 0.00765, 0.92750, 0.98781],
        [0.04308, 0.00705, 0.93426, 0.98621]], device='cuda:0')


In [31]:
print(probs)

None
